In [1]:
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import numpy as np
import random
import cv2
import os
import joblib
import albumentations as A
import imutils

random_seed = 42  
random.seed(random_seed)
np.random.seed(random_seed)
path_to_dataset = './dataset'


# Feature Extraction

Raw Pixels

In [2]:
def extract_raw_pixels(img):
    return img.flatten()

HOG

In [3]:
def extract_hog_features(img):
    
    win_size = (32, 32)
    cell_size = (4, 4)
    block_size_in_cells = (2, 2)

    block_size = (block_size_in_cells[1] * cell_size[1], block_size_in_cells[0] * cell_size[0])
    block_stride = (cell_size[1], cell_size[0])
    nbins = 9  # Number of orientation bins
    hog = cv2.HOGDescriptor(win_size, block_size, block_stride, cell_size, nbins)
    h = hog.compute(img)
    h = h.flatten()
    return h.flatten()

HSV Histogram

In [4]:
def extract_hsv_histogram(img, target_img_size=(128, 128)):
    """
    Extract an HSV histogram from an image.

    Parameters:
    - img: The input image in BGR format (as loaded by OpenCV).
    - target_img_size (tuple): The desired size to resize the image (width, height).

    Returns:
    - np.ndarray: The normalized and flattened HSV histogram.
    """
    hsv_img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

    hist = cv2.calcHist([hsv_img], channels=[0, 1, 2], mask=None, histSize=(8, 8, 8), ranges=[0, 180, 0, 256, 0, 256])

    if imutils.is_cv2():
        hist = cv2.normalize(hist)
    else:
        cv2.normalize(hist, hist)

    return hist.flatten()


# Augmentation

In [5]:
# transform = A.Compose([
#     A.HorizontalFlip(p=0.5),
#     A.VerticalFlip(p=0.5),
#     A.RandomRotate90(p=0.5),
#     A.OneOf([
#         A.GaussianBlur(p=0.2),
#         A.MotionBlur(p=0.2),
#         A.MedianBlur(blur_limit=3, p=0.2),
#     ], p=0.5),
#     A.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1, p=0.5),
#     A.RandomBrightnessContrast(p=0.5),
# ])

In [6]:
# import os
# import cv2
# import albumentations as A

# # Define augmentation pipeline
# augmentation_pipeline = A.Compose([
#     A.HorizontalFlip(p=0.2),
#     A.VerticalFlip(p=0.2),
#     A.RandomRotate90(p=0.2),
#     A.OneOf([
#         A.GaussianBlur(p=0.5),
#         A.MotionBlur(p=0.5),
#         A.MedianBlur(blur_limit=3, p=0.2),
#     ], p=0.5),
#     A.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1, p=0.5),
#     A.RandomBrightnessContrast(p=0.5),
# ])

# # Augment images and save in the same folder
# def augment_and_save_inplace(input_dir):
#     for dirpath, _, filenames in os.walk(input_dir):
#         for fn in filenames:
#             if not fn.lower().endswith(('.jpg', '.jpeg', '.png')):
#                 continue

#             # Read the image
#             img_path = os.path.join(dirpath, fn)
#             img = cv2.imread(img_path)
#             if img is None:
#                 print(f"Warning: Could not read image {img_path}")
#                 continue

#             # Perform augmentation
#             augmented = transform(image=img)['image']

#             # Generate new filename for the augmented image
#             name, ext = os.path.splitext(fn)
#             augmented_filename = f"{name}_aug{ext}"
#             augmented_path = os.path.join(dirpath, augmented_filename)

#             # Save the augmented image
#             cv2.imwrite(augmented_path, augmented)

# # Path to the dataset
# path_to_dataset = "./archive"

# # Augment and save
# augment_and_save_inplace(path_to_dataset)


# Loading Dataset

In [7]:
def load_dataset():
    features = {
        "raw": [],
        "hog": [],
        "hsv": []
    }
    labels = []
    labels_set = set()

    for dirpath, dirnames, filenames in os.walk(path_to_dataset):
        for fn in filenames:
            if not fn.lower().endswith(('.jpg', '.jpeg', '.png')):
                continue

            label = os.path.basename(dirpath)  # Use the folder name as the label
            labels.append(label)
            labels_set.add(label)

            path = os.path.join(dirpath, fn)
            img = cv2.imread(path)
            if img is None:
                print(f"Warning: Could not read image {path}")
                continue

            # img = cv2.resize(img, target_img_size)  # Resize to fixed size
            features["raw"].append(extract_raw_pixels(img))
            features["hog"].append(extract_hog_features(img))
            features["hsv"].append(extract_hsv_histogram(img))
        
    if not features:
        raise ValueError("No valid images were loaded. Please check the dataset path or image formats.")
    print(labels_set)
    print(len(labels_set))
    return features, labels

# Classification

In [8]:
classifiers = {
    'SVM': svm.LinearSVC(random_state=random_seed),
    'KNN': KNeighborsClassifier(n_neighbors=7, n_jobs=-1),
    'RandomForest': RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
}

def classify_image(img, model_name, feature):

    # Load the trained model
    model = joblib.load(f"./Models/{model_name}_{feature}_model.pkl")
    
    # Convert image to grayscale if needed and resize to match training input size
    # img = cv2.resize(img, target_img_size)  # Uncomment if resizing was used during training

    # Extract raw pixel features and reshape to match the training input format
    if feature == "raw":
        img_features = extract_raw_pixels(img).reshape(1, -1)
    elif feature == "hog":
        img_features = extract_hog_features(img).reshape(1, -1)
    elif feature == "hsv":
        img_features = extract_hsv_histogram(img).reshape(1, -1)

    # Predict the label
    label = model.predict(img_features)

    # print(f"Predicted label: {label[0]}")
    return label[0]

# Training

In [9]:
def train(features, labels):

    train_features = {
        "raw": None,
        "hog": None,
        "hsv": None
    }

    test_features = {
        "raw": None,
        "hog": None,
        "hsv": None
    }

    train_labels = {
        "raw": None,
        "hog": None,
        "hsv": None
    }

    test_labels = {
        "raw": None,
        "hog": None,
        "hsv": None
    }
    
    train_features["raw"], test_features["raw"], train_labels["raw"], test_labels["raw"] = train_test_split(
        features["raw"], labels, test_size=0.3, random_state=random_seed)
    
    train_features["hog"], test_features["hog"], train_labels["hog"], test_labels["hog"] = train_test_split(
        features["hog"], labels, test_size=0.3, random_state=random_seed)
    
    train_features["hsv"], test_features["hsv"], train_labels["hsv"], test_labels["hsv"] = train_test_split(
        features["hsv"], labels, test_size=0.3, random_state=random_seed)
    
    for model_name, model in classifiers.items():
        print('############## Training', model_name, "##############")
        for feature, label in zip(features.keys(), train_labels.keys()):
            model.fit(train_features[feature], train_labels[label])

            accuracy = model.score(test_features[feature], test_labels[label])

            print(model_name, 'with', f"{feature} features", 'accuracy:', accuracy*100, '%')
            # Save the trained model
            joblib.dump(model, f"./Models/{model_name}_{feature}_model.pkl")

            print(f"{model_name}_{feature} model saved successfully.")
        print()

In [10]:
print('Loading dataset. This will take time ...')
features, labels = load_dataset()
print('Finished loading dataset.')

Loading dataset. This will take time ...
{'L', 'P', 'V', 'R', 'E', 'H', 'O', 'Q', 'W', 'Z', 'A', 'X', 'C', 'F', 'D', 'B', 'N', 'J', 'I', 'S', 'U', 'T', 'M', 'Y', 'K', 'G'}
26
Finished loading dataset.


In [11]:
train(features, labels)

############## Training SVM ##############
SVM with raw features accuracy: 71.90243902439025 %
SVM_raw model saved successfully.
SVM with hog features accuracy: 91.70731707317074 %
SVM_hog model saved successfully.
SVM with hsv features accuracy: 10.975609756097562 %
SVM_hsv model saved successfully.

############## Training KNN ##############
KNN with raw features accuracy: 81.17073170731707 %
KNN_raw model saved successfully.
KNN with hog features accuracy: 93.46341463414635 %
KNN_hog model saved successfully.
KNN with hsv features accuracy: 12.341463414634147 %
KNN_hsv model saved successfully.

############## Training RandomForest ##############
RandomForest with raw features accuracy: 83.7560975609756 %
RandomForest_raw model saved successfully.
RandomForest with hog features accuracy: 90.09756097560975 %
RandomForest_hog model saved successfully.
RandomForest with hsv features accuracy: 15.121951219512194 %
RandomForest_hsv model saved successfully.



In [12]:
from skimage import io
from skimage.util import invert

img = io.imread('test.jpg')
inverted_img = invert(img)

classify_image(inverted_img, "RandomForest", "hog")

'R'